# Lezione 23 - DBSCAN: clustering basato su densita

## Sezione 1 - Titolo e obiettivi

Obiettivo: usare DBSCAN per trovare cluster di forma arbitraria, scegliere eps e min_samples, e gestire outlier (noise).

### Cosa impari
- Differenze tra clustering basato su densita e su distanza.
- Ruolo di eps e min_samples e loro scelta con k-distance graph.
- Core/border/noise e come leggerli nei risultati.
- Confronto DBSCAN vs K-Means vs Gerarchico.

### Perche serve
Quando le forme non sono sferiche o ci sono outlier, K-Means e Gerarchico possono fallire; DBSCAN trova cluster per densita e marca il rumore invece di forzare assegnazioni.

### Prerequisiti minimi
- Conoscere K-Means e clustering gerarchico (lezioni precedenti).
- Saper usare StandardScaler e silhouette score.
- Nozioni di distanza euclidea.


## Sezione 2 - Teoria profonda

### 1.1 Perche DBSCAN
- Non richiede K a priori: scopre automaticamente quanti cluster ci sono.
- Trova cluster di forma arbitraria (anche a mezzaluna).
- Identifica outlier (label = -1) invece di forzare assegnazioni.
- Limite: soffre quando le densita tra cluster sono molto diverse.


### 1.2 Parametri chiave: eps e min_samples

| Parametro | Significato | Effetto pratico |
|-----------|-------------|-----------------|
| eps | Raggio del vicinato | eps basso -> piu noise, eps alto -> cluster fusi |
| min_samples | Min punti nel vicinato per essere denso | Valore basso sensibile al rumore, alto richiede zone molto dense |

Regola empirica: `min_samples >= D + 1` (D = numero di feature). In 2D parti da 4-5.


### 1.3 Tipi di punti e regola di costruzione del cluster

- Core point: ha almeno min_samples punti entro eps.
- Border point: non e core, ma e entro eps da un core.
- Noise point: non e core e non e vicino a nessun core (label = -1).

Un cluster e l'insieme dei core connessi (entro eps) piu i border collegati a quei core.


### 1.4 Algoritmo in breve

1) Visita un punto non etichettato.
2) Trova i vicini entro eps.
3) Se i vicini sono meno di min_samples -> noise provvisorio.
4) Altrimenti diventa core, crea un cluster, espandi aggiungendo vicini (visitandoli) finche non ci sono nuovi core connessi.
5) Alla fine i noise provvisori non collegati restano label -1.

Complessita: con strutture di vicinato (KD-tree/Ball-tree) circa O(n log n); senza, O(n^2).


### 1.5 Scegliere eps: k-distance graph

Procedura:
1. Scegli min_samples (almeno D+1).
2. Calcola per ogni punto la distanza al k-esimo vicino (k = min_samples).
3. Ordina queste distanze e plottale.
4. Il gomito del grafico e un buon eps: sotto il gomito cluster compatti, sopra il gomito unisce cluster.

Se eps e troppo piccolo: tanti noise e cluster spezzati. Se eps e troppo grande: pochi cluster e rumore assorbito.


## Sezione 3 - Schema mentale e decision map

Workflow DBSCAN (dopo lo scaling):

```
StandardScaler -> stima min_samples (>= D+1)
              -> k-distance graph -> scegli eps al gomito
              -> DBSCAN(eps, min_samples)
              -> conta cluster (escludendo label -1)
              -> valuta: % noise, silhouette (senza noise), forme plausibili
              -> tuning: aumenta eps se troppo noise; riduci eps se cluster fusi
```

Checklist rapida
- [ ] Dati scalati.
- [ ] min_samples motivato (>= D+1).
- [ ] eps stimato da k-distance graph.
- [ ] Gestione noise: calcolata percentuale e interpretata.
- [ ] Silhouette calcolata escludendo label -1.


## Sezione 4 - Notebook dimostrativo

### Perche questo passo (Demo 1 - Moons)
Mostriamo come DBSCAN separa forme non convessa dove K-Means fallisce. Ci aspettiamo che K-Means tagli le mezzalune, mentre DBSCAN riconosce i due archi e isola gli outlier.


### Checkpoint operativi per le demo
- Import completi in ogni demo per eseguire cellule singole.
- Assert su shape e assenza di NaN dopo lo scaling.
- Silhouette calcolata solo sui punti non noise (label != -1).


In [ ]:
# Demo 1 - DBSCAN su moons vs K-Means (forme non convesse)
# Intento: mostrare che DBSCAN gestisce forme a mezzaluna mentre K-Means le taglia male.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.datasets import make_moons

np.random.seed(42)
X, y_true = make_moons(n_samples=300, noise=0.05)
assert X.ndim == 2 and not np.isnan(X).any(), "Dati malformati"

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
assert X_scaled.shape == X.shape

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
axes[0].scatter(X[:, 0], X[:, 1], c=y_true, cmap='viridis', s=30, alpha=0.7)
axes[0].set_title('Ground truth')
axes[0].set_xlabel('x'); axes[0].set_ylabel('y'); axes[0].grid(True, alpha=0.3)

kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
labels_km = kmeans.fit_predict(X_scaled)
ari_km = adjusted_rand_score(y_true, labels_km)
axes[1].scatter(X[:, 0], X[:, 1], c=labels_km, cmap='viridis', s=30, alpha=0.7)
axes[1].set_title(f'K-Means (ARI={ari_km:.3f})')
axes[1].grid(True, alpha=0.3)

dbscan = DBSCAN(eps=0.3, min_samples=5)
labels_db = dbscan.fit_predict(X_scaled)
ari_db = adjusted_rand_score(y_true, labels_db)
noise_mask = labels_db == -1
colors = labels_db.astype(float)
colors[noise_mask] = -0.5
axes[2].scatter(X[:, 0], X[:, 1], c=colors, cmap='viridis', s=30, alpha=0.7)
if noise_mask.sum() > 0:
    axes[2].scatter(X[noise_mask, 0], X[noise_mask, 1], c='red', marker='x', s=50, label='Noise')
    axes[2].legend()
axes[2].set_title(f'DBSCAN (ARI={ari_db:.3f})')
axes[2].grid(True, alpha=0.3)

plt.tight_layout(); plt.show()

n_clusters = len(set(labels_db)) - (1 if -1 in labels_db else 0)
print(f"Cluster trovati da DBSCAN: {n_clusters}, noise: {noise_mask.sum()} punti")
print("Nota: silhouette va calcolata escludendo label -1 se serve una metrica di qualita.")


### Perche questo passo (Demo 2 - k-distance graph)
Usiamo il k-distance graph per stimare eps, includendo outlier. Obiettivo: vedere un gomito chiaro e usare quel valore per DBSCAN.


In [ ]:
# Demo 2 - k-distance graph per stimare eps
# Intento: usare il gomito del k-distance graph per scegliere eps e applicare DBSCAN.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_blobs
from sklearn.neighbors import NearestNeighbors

np.random.seed(42)
X_blobs, _ = make_blobs(n_samples=250, centers=3, cluster_std=0.8, random_state=42)
outliers = np.random.uniform(low=-10, high=10, size=(15, 2))
X_full = np.vstack([X_blobs, outliers])
scaler = StandardScaler(); X_scaled = scaler.fit_transform(X_full)
assert not np.isnan(X_scaled).any()

min_samples = 5
nn = NearestNeighbors(n_neighbors=min_samples)
nn.fit(X_scaled)
distances, _ = nn.kneighbors(X_scaled)
k_dist = np.sort(distances[:, -1])

# Knee via seconda derivata
second_der = np.diff(np.diff(k_dist))
knee_idx = int(np.argmax(second_der) + 2)
eps_opt = float(k_dist[knee_idx])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].plot(range(len(k_dist)), k_dist, 'b-', linewidth=2)
axes[0].axhline(y=eps_opt, color='red', linestyle='--', label=f'eps ~ {eps_opt:.2f}')
axes[0].scatter([knee_idx], [eps_opt], color='red', s=80)
axes[0].set_title('k-distance graph (gomito)'); axes[0].grid(True, alpha=0.3)
axes[0].legend()

model = DBSCAN(eps=eps_opt, min_samples=min_samples)
labels = model.fit_predict(X_scaled)
noise_mask = labels == -1
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

colors = plt.cm.viridis(np.linspace(0, 1, n_clusters + 1))
for i in range(n_clusters):
    mask = labels == i
    axes[1].scatter(X_full[mask, 0], X_full[mask, 1], c=[colors[i]], s=40, alpha=0.7, label=f'Cluster {i}')
axes[1].scatter(X_full[noise_mask, 0], X_full[noise_mask, 1], c='red', marker='x', s=60, label='Noise')
axes[1].set_title(f'DBSCAN con eps stimato Cluster={n_clusters}, Noise={noise_mask.sum()}'); axes[1].legend(); axes[1].grid(True, alpha=0.3)
plt.tight_layout(); plt.show()


### Perche questo passo (Demo 3 - Core, border, noise)
Visualizziamo come DBSCAN etichetta core, border e noise su un dataset piccolo. Obiettivo: leggere le categorie e la percentuale di rumore.


In [ ]:
# Demo 3 - Core, border, noise
# Intento: evidenziare i tre tipi di punti prodotti da DBSCAN.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

np.random.seed(123)
X_small = np.array([
    [1,1],[1.2,0.8],[0.8,1.2],[1.1,1.1],[0.9,0.9],[1.3,1.0],[1.0,1.3],
    [4,4],[4.2,3.8],[3.8,4.2],[4.1,4.1],[3.9,3.9],
    [8,1],[8.1,1.2],[7.9,0.8],
    [6,6]  # outlier isolato
])
scaler = StandardScaler(); X_scaled = scaler.fit_transform(X_small)

model = DBSCAN(eps=0.6, min_samples=3)
labels = model.fit_predict(X_scaled)
core_mask = np.zeros_like(labels, dtype=bool)
core_mask[model.core_sample_indices_] = True
noise_mask = labels == -1

fig, ax = plt.subplots(figsize=(7,5))
ax.scatter(X_small[core_mask,0], X_small[core_mask,1], c='blue', s=120, edgecolors='black', label='Core')
ax.scatter(X_small[~core_mask & ~noise_mask,0], X_small[~core_mask & ~noise_mask,1], c='green', s=80, edgecolors='black', label='Border')
ax.scatter(X_small[noise_mask,0], X_small[noise_mask,1], c='red', marker='x', s=120, label='Noise')
ax.set_title('Core vs Border vs Noise'); ax.grid(True, alpha=0.3); ax.legend()
plt.tight_layout(); plt.show()

print(f"Cluster trovati: {len(set(labels)) - (1 if -1 in labels else 0)}; Noise: {noise_mask.sum()} punti")


### Perche questo passo (Demo 4 - Effetto eps e min_samples)
Variare eps e min_samples per capire sensibilita e stabilita dei cluster e del noise. Obiettivo: mostrare come tuning influenza risultati e silhouette.


In [ ]:
# Demo 4 - Effetto di eps e min_samples
# Intento: mostrare come variazioni dei parametri cambiano cluster e noise.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_circles
from sklearn.metrics import silhouette_score

np.random.seed(42)
X_circ, _ = make_circles(n_samples=400, factor=0.5, noise=0.05)
X_scaled = StandardScaler().fit_transform(X_circ)
assert not np.isnan(X_scaled).any()

parametri = [
    {'eps':0.15, 'min_samples':3},
    {'eps':0.25, 'min_samples':4},
    {'eps':0.35, 'min_samples':6}
]

fig, axes = plt.subplots(1, 3, figsize=(15,5))
for ax, pars in zip(axes, parametri):
    db = DBSCAN(eps=pars['eps'], min_samples=pars['min_samples'])
    labels = db.fit_predict(X_scaled)
    noise_mask = labels == -1
    mask = labels != -1
    sil = silhouette_score(X_scaled[mask], labels[mask]) if mask.sum()>1 and len(set(labels[mask]))>1 else -1
    ax.scatter(X_circ[mask,0], X_circ[mask,1], c=labels[mask], cmap='viridis', s=20, alpha=0.7)
    ax.scatter(X_circ[noise_mask,0], X_circ[noise_mask,1], c='red', marker='x', s=30, label='Noise')
    ax.set_title(f"eps={pars['eps']}, min_samples={pars['min_samples']} Silhouette={sil:.3f}, Noise={noise_mask.sum()}")
    ax.grid(True, alpha=0.3)
plt.tight_layout(); plt.show()

print("Tuning: aumenta eps per ridurre noise; diminuisci eps o aumenta min_samples se cluster si fondono troppo.")


### Perche questo passo (Demo 5 - Confronto finale)
Confrontiamo DBSCAN con K-Means e Agglomerative su forme complesse per scegliere il metodo piu adatto in base a forma, noise e silhouette.


In [ ]:
# Demo 5 - Confronto DBSCAN vs K-Means vs Agglomerative
# Intento: scegliere l'algoritmo in base a forma e noise.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.metrics import adjusted_rand_score
from sklearn.datasets import make_moons, make_circles

np.random.seed(42)
X1, y1 = make_moons(n_samples=300, noise=0.07)
X2, y2 = make_circles(n_samples=200, noise=0.05, factor=0.5)
X = np.vstack([X1, X2]); y_true = np.concatenate([y1, y2 + 2])
X_scaled = StandardScaler().fit_transform(X)

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
labels_km = kmeans.fit_predict(X_scaled)
ari_km = adjusted_rand_score(y_true, labels_km)

agg = AgglomerativeClustering(n_clusters=4, linkage='ward')
labels_ag = agg.fit_predict(X_scaled)
ari_ag = adjusted_rand_score(y_true, labels_ag)

db = DBSCAN(eps=0.25, min_samples=5)
labels_db = db.fit_predict(X_scaled)
ari_db = adjusted_rand_score(y_true, labels_db)
noise_mask = labels_db == -1

fig, axes = plt.subplots(1, 3, figsize=(15,4))
axes[0].scatter(X[:,0], X[:,1], c=labels_km, cmap='viridis', s=15, alpha=0.7)
axes[0].set_title(f'K-Means (ARI={ari_km:.3f})'); axes[0].grid(True, alpha=0.3)
axes[1].scatter(X[:,0], X[:,1], c=labels_ag, cmap='viridis', s=15, alpha=0.7)
axes[1].set_title(f'Agglomerative (ARI={ari_ag:.3f})'); axes[1].grid(True, alpha=0.3)
colors = labels_db.astype(float); colors[noise_mask] = -0.5
axes[2].scatter(X[:,0], X[:,1], c=colors, cmap='viridis', s=15, alpha=0.7)
axes[2].set_title(f'DBSCAN (ARI={ari_db:.3f}, noise={noise_mask.sum()})'); axes[2].grid(True, alpha=0.3)
plt.tight_layout(); plt.show()

print("Confronto: DBSCAN gestisce forme e noise; K-Means assume sfericita; Agglomerativo dipende dal linkage scelto.")


## Sezione 5 - Esercizi guidati (step by step)

### Perche questo esercizio (23.1)
Usare DBSCAN per anomaly detection su transazioni, scegliendo eps con k-distance graph e interpretando le anomalie.


In [ ]:
# Esercizio 23.1 - Anomaly detection con DBSCAN
# Intento: individuare transazioni sospette usando k-distance graph e DBSCAN.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

importo = [25,30,28,35,500,22,27,1000,33,29,31,26,750,28,24]
durata = [5,6,4,7,2,5,6,1,8,5,6,4,3,5,6]

df = pd.DataFrame({'transazione':[f'T{i}' for i in range(1,16)], 'importo':importo, 'durata_min':durata})
X = df[['importo','durata_min']].values
scaler = StandardScaler(); Xs = scaler.fit_transform(X)
assert not np.isnan(Xs).any()

min_samples = 3
nn = NearestNeighbors(n_neighbors=min_samples)
nn.fit(Xs)
dist, _ = nn.kneighbors(Xs)
k_dist = np.sort(dist[:, -1])
second_der = np.diff(np.diff(k_dist))
knee = int(np.argmax(second_der) + 2) if len(second_der)>0 else len(k_dist)//2
eps_opt = float(k_dist[knee])

fig, axes = plt.subplots(1,2, figsize=(13,5))
axes[0].plot(range(len(k_dist)), k_dist, 'b-o'); axes[0].axhline(y=eps_opt, color='red', linestyle='--', label=f'eps~{eps_opt:.2f}')
axes[0].set_title('k-distance graph'); axes[0].legend(); axes[0].grid(True, alpha=0.3)

db = DBSCAN(eps=max(eps_opt,0.6), min_samples=min_samples)
labels = db.fit_predict(Xs)
df['cluster'] = labels; df['anomalia'] = labels == -1
colors = ['red' if a else 'green' for a in df['anomalia']]
axes[1].scatter(df['importo'], df['durata_min'], c=colors, s=120, edgecolors='black')
for _, row in df.iterrows():
    axes[1].annotate(row['transazione'], (row['importo']+10, row['durata_min']+0.1), fontsize=8)
axes[1].set_title(f"DBSCAN eps~{max(eps_opt,0.6):.2f}, noise={df['anomalia'].sum()}")
axes[1].grid(True, alpha=0.3)
plt.tight_layout(); plt.show()

print(df[['transazione','importo','durata_min','cluster','anomalia']])


### Perche questo esercizio (23.2)
Applicare DBSCAN a coordinate geografiche per individuare zone dense e POI isolati senza imporre K.


In [ ]:
# Esercizio 23.2 - Clustering geografico di POI
# Intento: raggruppare punti di interesse vicini e trovare isolati.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

lat = [45.46,45.47,45.46,45.48,45.70,45.71,45.69,45.20,45.46,45.47]
lon = [9.18,9.19,9.17,9.18,9.30,9.31,9.29,9.50,9.20,9.18]
nomi = ['Duomo','Scala','Castello','Brera','Monza1','Monza2','Monza3','Pavia','Navigli','Porta Romana']

df_poi = pd.DataFrame({'nome':nomi,'lat':lat,'lon':lon})
X_geo = df_poi[['lat','lon']].values
assert X_geo.shape[1]==2

min_samples = 2
eps_geo = 0.05  # circa 5km in coordinate decimali

nn = NearestNeighbors(n_neighbors=min_samples)
nn.fit(X_geo); dist,_ = nn.kneighbors(X_geo); k_dist=np.sort(dist[:,-1])

fig, axes = plt.subplots(1,2, figsize=(14,5))
axes[0].plot(range(len(k_dist)), k_dist, 'b-o'); axes[0].axhline(y=eps_geo, color='red', linestyle='--', label=f'eps={eps_geo}')
axes[0].set_title('k-distance graph (geo)'); axes[0].legend(); axes[0].grid(True, alpha=0.3)

db = DBSCAN(eps=eps_geo, min_samples=min_samples)
labels = db.fit_predict(X_geo)
df_poi['cluster'] = labels
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

colors = plt.cm.Set1(np.linspace(0,1,n_clusters+1))
for cid in range(n_clusters):
    mask = labels==cid
    axes[1].scatter(df_poi.loc[mask,'lon'], df_poi.loc[mask,'lat'], c=[colors[cid]], s=160, edgecolors='black', alpha=0.8, label=f'Zona {cid}')
noise_mask = labels==-1
if noise_mask.sum()>0:
    axes[1].scatter(df_poi.loc[noise_mask,'lon'], df_poi.loc[noise_mask,'lat'], c='gray', s=180, marker='X', edgecolors='black', label='Isolati')
for _, row in df_poi.iterrows():
    axes[1].annotate(row['nome'], (row['lon']+0.01, row['lat']+0.01), fontsize=9)
axes[1].set_title(f'Clustering POI (cluster={n_clusters}, isolati={noise_mask.sum()})'); axes[1].legend(); axes[1].grid(True, alpha=0.3)
plt.tight_layout(); plt.show()

print(df_poi[['nome','lat','lon','cluster']])


### Perche questo esercizio (23.3)
Implementare un tuning automatico di eps e confrontarlo con scelte manuali, valutando silhouette e noise.


In [ ]:
# Esercizio 23.3 - Tuning automatico di eps
# Intento: stimare eps con gomito e confrontarlo con scelte manuali.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.datasets import make_moons
from sklearn.neighbors import NearestNeighbors

np.random.seed(42)
X, y_true = make_moons(n_samples=300, noise=0.08)
Xs = StandardScaler().fit_transform(X)
min_samples = 5

def trova_eps_ottimale(X, min_samples, plot=False):
    nn = NearestNeighbors(n_neighbors=min_samples)
    nn.fit(X)
    dist,_ = nn.kneighbors(X)
    k_dist = np.sort(dist[:,-1])
    if len(k_dist)>3:
        second_der = np.diff(np.diff(k_dist))
        knee = int(np.argmax(second_der) + 2)
    else:
        knee = len(k_dist)//2
    eps = float(k_dist[knee])
    if plot:
        plt.figure(figsize=(7,4))
        plt.plot(range(len(k_dist)), k_dist, 'b-')
        plt.axhline(y=eps, color='red', linestyle='--', label=f'eps={eps:.3f}')
        plt.scatter([knee],[eps], color='red'); plt.legend(); plt.title('k-distance graph'); plt.grid(True, alpha=0.3); plt.show()
    return eps

eps_auto = trova_eps_ottimale(Xs, min_samples, plot=True)
print(f"eps ottimale stimato: {eps_auto:.3f}")

eps_values = [0.1, 0.3, 0.5, eps_auto]
labels_text = ['0.1', '0.3', '0.5', 'auto']
fig, axes = plt.subplots(2,2, figsize=(12,10)); axes = axes.flatten()
results = {}
for ax, eps, lab in zip(axes, eps_values, labels_text):
    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels = db.fit_predict(Xs)
    mask = labels != -1
    sil = silhouette_score(Xs[mask], labels[mask]) if mask.sum()>1 and len(set(labels[mask]))>1 else -1
    ari = adjusted_rand_score(y_true, labels)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = (labels==-1).sum()
    colors = labels.astype(float); colors[labels==-1] = -0.5
    ax.scatter(X[:,0], X[:,1], c=colors, cmap='viridis', s=20, alpha=0.7)
    if n_noise>0:
        ax.scatter(X[labels==-1,0], X[labels==-1,1], c='red', marker='x', s=30, label='Noise')
        ax.legend()
    ax.set_title(f"eps={lab} K={n_clusters}, noise={n_noise}, sil={sil:.3f}, ari={ari:.3f}")
    ax.grid(True, alpha=0.3)
    results[lab] = {'eps':eps, 'sil':sil, 'ari':ari, 'noise':n_noise, 'clusters':n_clusters}
plt.tight_layout(); plt.show()

print("eps  | clusters | noise | silhouette | ARI")
for lab, r in results.items():
    print(f"{lab:<4} {r['clusters']:>8} {r['noise']:>7} {r['sil']:>11.3f} {r['ari']:>8.3f}")
print("Conclusione: il gomito fornisce un buon eps, ma puo richiedere ritocco manuale.")


## Sezione 6 - Conclusione operativa

### Cosa portarsi a casa
- DBSCAN lavora per densita: non richiede K e identifica noise (-1) invece di forzare assegnazioni.
- eps si sceglie con il k-distance graph; min_samples tipicamente >= dimensioni + 1.
- Calcola silhouette escludendo i noise; valuta anche % noise e forme ottenute.
- Se i cluster hanno densita molto diverse, DBSCAN puo fallire: considera alternative o segmenta per zona.

### Methods explained (uso, input/output, errori tipici)
- `StandardScaler`: standardizza feature (input array n_samples x n_features, output stessa shape); errori: NaN o colonne costanti; sempre prima di distanze.
- `NearestNeighbors`: trova vicini per k-distance graph; input X 2D, n_neighbors int; output distanze array; errori: n_neighbors troppo grande rispetto ai campioni.
- `DBSCAN`: clustering per densita; input X 2D, eps float, min_samples int; output labels 1D con -1 per noise; errori: eps troppo basso (tutto noise) o troppo alto (cluster fusi).
- `silhouette_score`: qualita di separazione; input X 2D, labels 1D; output float; errore: include noise -> filtrare label -1.
- `adjusted_rand_score`: confronto con verita terreno; input y_true, y_pred; output [-1,1]; errore: ignorare noise se compari a ground truth pulita.

### Common errors and quick debug
- Dati non scalati: eps sbilanciato; soluzione: StandardScaler.
- Uso di silhouette con noise incluso: valori ingannevoli; soluzione: filtra label -1.
- eps scelto a occhio senza k-distance: cluster instabili; soluzione: usa grafico e annota gomito.
- min_samples troppo basso: troppi punti core e poco noise; soluzione: aumenta a >= D+1.


## Sezione 7 - End-of-lesson checklist e glossario

### Checklist finale
- [ ] Dati scalati con `StandardScaler`.
- [ ] min_samples scelto (>= dimensioni + 1) e motivato.
- [ ] k-distance graph calcolato e eps scelto al gomito.
- [ ] DBSCAN eseguito e percentuale di noise valutata.
- [ ] Silhouette calcolata escludendo noise.
- [ ] Cluster interpretati e, se necessario, confrontati con K-Means/Gerarchico.

### Glossario (termini usati)
- eps: raggio del vicinato.
- min_samples: punti minimi per densita.
- Core point: punto con abbastanza vicini entro eps.
- Border point: vicino a un core ma non core.
- Noise: punti etichettati -1 (fuori da cluster).
- k-distance graph: grafico delle distanze al k-esimo vicino per stimare eps.
- Silhouette: metrica di separazione tra cluster.
- ARI (Adjusted Rand Index): confronto tra clustering e verita a terra.
- StandardScaler: standardizzazione feature.
- NearestNeighbors: ricerca dei vicini piu prossimi.
- DBSCAN: algoritmo di clustering basato su densita.

## Sezione 8 - Didactic changelog (max 10 voci)
1. Riorganizzate le 8 sezioni con obiettivi, teoria, schema mentale, demo ed esercizi guidati.
2. Ripulito testo da emoji e portato a ASCII, aggiungendo rationales pre-demo/esercizi.
3. Inseriti Methods explained, errori comuni, checklist e glossario nelle sezioni finali.
4. Commentati e rinforzati i code cell con assert su shape/NaN e note su silhouette senza noise.
